In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [ ]:
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.ontology import Graph, OntologyManager, OntologyConfig
from backend.datasetmatcher import DatasetManager
from backend.explorative.explorative_support import GuidanceManager
from backend.explorative.llm_query import LLMQuery
from backend.explorative.topic_init import TopicInitator

from backend.eval_config import (
    DBPEDIA_CONFIGS,
    OMA_CONFIGS,
    UNIPROT_CONFIGS,
    BTO_CONFIGS,
    DNB_CONFIGS,
    OLYMPICS_CONFIGS,
    EvalConfig,
)
base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

# store = SPARQLStore(
#     "http://localhost:7200/repositories/dpedia",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )

# store = SPARQLStore(
#     "http://localhost:3030/dbpedia/query",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )
setup: EvalConfig = BTO_CONFIGS[-1]
print("Setup for ", setup.name)
store = SPARQLStore(
    setup.sparql_endpoint,
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
    retries=10,
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


In [ ]:

guidance_man = GuidanceManager(
    ontology_manager, conn_str=setup.conn_str, llm_model_id=setup.model_id
)
print(guidance_man.identifier)
llm_query = LLMQuery(guidance_man)
topic_init = TopicInitator(guidance_man)

In [ ]:
topic_init.initate()

In [14]:
topic_init.model_topics()

Building documents:  29%|██▉       | 68/234 [00:21<01:55,  1.44it/s]

ignoring <https://w3id.org/brainteaser/ontology/schema/MuscleTone>


Building documents: 100%|██████████| 234/234 [02:59<00:00,  1.31it/s]
2025-06-05 10:21:10,148 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/195 [00:00<?, ?it/s]

2025-06-05 10:21:13,497 - BERTopic - Embedding - Completed ✓
2025-06-05 10:21:13,498 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-05 10:21:15,347 - BERTopic - Dimensionality - Completed ✓
2025-06-05 10:21:15,347 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-05 10:21:15,421 - BERTopic - Cluster - Completed ✓
2025-06-05 10:21:15,424 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/90 [00:00<?, ?it/s]Llama.generate: 267 prefix-match hit, remaining 209 prompt tokens to eval
llama_perf_context_print:        load time =      24.17 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   209 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    10 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      74.06 ms /   219 tokens
Llama.generate

In [ ]:
llm_query.initiate(n_queries=10, force=True)